Traducción al español de http://www.astropy.org/astropy-tutorials/Coordinates.html hecha por Germán Chaparro.

# Usando `coordinates` y `table` para combinar y comparar catálogos

In [ ]:
import urllib
import IPython.display

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table
import astropy.coordinates as coord

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
hcg7_center = SkyCoord.from_name('Hcg 7')
hcg7_center

Al final de la sección anterior, encontramos que HCG7 está en SDSS, lo que significa que podemos descargar catálogos de objetos directamente desde SDSS. Más adelante, combinaremos este catálogo con otro catálogo que cubra el mismo campo, permitiéndonos hacer gráficas combinando ambos catálogos.

Vamos a entrar a la base de datos SQL de SDSS usando el paquete [astroquery](https://astroquery.readthedocs.org).  This will require an internet connection and a working install of astroquery. Si usted no tiene este paquete, puede ignorar las próximas dos celdas, porque los archivos de datos están en el repositorio. Dependiendo de su versión de `astroquery`, puede que aparezca un Warning, que podemos ignorar sin problema.

In [ ]:
from astroquery.sdss import SDSS
sdss = SDSS.query_region(coordinates=hcg7_center, radius=20*u.arcmin, 
                         spectro=True, 
                         photoobj_fields=['ra','dec','u','g','r','i','z'])

Los *queries* hechos a `astroquery` nos devuelve un objeto [`astropy.table.Table`](http://docs.astropy.org/en/stable/table/index.html).  Podemos trabajar con este objeto sin guardar nada en disco, pero en esta ocasión si lo haremos. De esta manera, al cerrar la sesión y volver, usted no tiene que hacer un *query* de nuevo.

(Esto no funcionará si usted no corrió la celda anterior. No hay problema, simplemente vaya a la celda que tiene ``Table.read`` y use la copia de esta tabla, incluida en el repositorio.)

In [ ]:
sdss.write('HCG7_SDSS_photo.dat', format='ascii')

Si usted no tiene internet, simplemente puede leer la tabla desde Python corriendo la siguiente celda. Si usted hizo el query anterior, usted puede ignorar esta instrucción, dado que la tabla ya está almacenada en memoria como la variable llamada `sdss`.

In [ ]:
sdss = Table.read('HCG7_SDSS_photo.dat', format='ascii')

Ahora tenemos un catálogo de objetos que obtuvimos de SDSS. Supongamos que usted tiene su propio catálogo de objetos en el mismo campo para compararlo con el de SDSS. En este caso usaremos un catálogo extraído de [2MASS](http://www.ipac.caltech.edu/2mass/).  Vamos a cargar este catálogo (que está en el repositorio) a Python.

In [ ]:
twomass = Table.read('HCG7_2MASS.tbl', format='ascii')

Para combinar los catálogos, necesitamos objetos `SkyCoord`. Vamos a contruirlos a partir de las tablas que cargamos. Esto resulta ser directo: tomamos las columnas `ra` y `dec` de la tabla y se las damos al constructor `SkyCoord` constructor.  Primero hagamos una inspección de las tablas.

In [ ]:
sdss[0:5]

In [ ]:
twomass[0:5]

Ya que tenemos las columnas de `ra` y `dec` podemos usarlas para crear nuestros `SkyCoord`s.

No es necesario crear un `SkyCoord` para cada fila en la tabla. En vez de esto, aprovechamos que `SkyCoord` recibe *arrays* de valores de coordenadas, ya sean `Quantity`s, listas de *strings*, columnas de `Table`s, etc., y `SkyCoord` hará las operaciones tranquilamente elemento a elemento.

In [ ]:
coo_sdss = SkyCoord(sdss['ra']*u.deg, sdss['dec']*u.deg)
coo_twomass = SkyCoord(twomass['ra'], twomass['dec'])

In [ ]:
coo_twomass

Hay una diferencia sutil. Para SDSS tuvimos que dar unidades, pero no para 2MASS. Esto es porque la tabla de 2MASS tiene unidades asociadas a las columnas, mientras que SDSS no tiene unidades.

Ahora usamos el método ``SkyCoord.match_to_catalog_sky`` para combinar ambos catálogos. El orden importa: combinamos 2MASS sobre SDSS porque hay muchas más entradas en SDSS, de manera que es probable que casi todos los objetos de 2MASS están en SDSS y no al revés.

In [ ]:
idx_sdss, d2d_sdss, d3d_sdss = coo_twomass.match_to_catalog_sky(coo_sdss)

``idx`` da los índices de ``coo_sdss`` que se ajustan mejor a SDSS, mientras ``d2d`` y ``d3d`` son las distancias en cielo y en espacio real entre posibles coincidencias. En nuestro caso ignoraremos ``d3d`` porque no dimos información sobre la distancia a lo largo de la línea de visión. En cambio ``d2d`` nos da un buen diagnóstico de las posibles coincidencias:

In [ ]:
plt.hist(d2d_sdss.arcsec, histtype='step', range=(0,2))
plt.xlabel('Separation [arcsec]')
plt.tight_layout()

Se ve bien, pues todas las posibles coincidencias están dentro de un segundo de arco.

Ahora podemos calcular cosas como colores que combinen la fotometría de SDSS y 2MASS.

In [ ]:
rmag = sdss['r'][idx_sdss]
grcolor = sdss['g'][idx_sdss] - rmag
rKcolor = rmag - twomass['k_m_ext']

plt.subplot(1, 2, 1)
plt.scatter(rKcolor, rmag)
plt.xlabel('r-K')
plt.ylabel('r')
plt.xlim(2.5, 4)
plt.ylim(18, 12) #mags go backwards!

plt.subplot(1, 2, 2)
plt.scatter(rKcolor, rmag)
plt.xlabel('r-K')
plt.ylabel('g-r')
plt.xlim(2.5, 4)
plt.tight_layout()

Ok, todos están dentro de un arcosegundo, lo cual es prometedor. Pero, ¿estamos seguros de que no es sólo que cualquier objeto tendría coincidencias dentro de un segundo de arco? Vamos a comprobarlo comparando con un conjunto de puntos aleatorios.

Primero creamos un conjunto de puntos uniformemente aleatorios (con un tamaño que coincida con `coo_twomass`) que cubran el mismo rango de RA/Decs que hay en `coo_sdss`. Esto lo haremos con el método `ptp()` (peak-to-peak).

In [ ]:
ras_sim = np.random.rand(len(coo_twomass))*coo_sdss.ra.ptp() + coo_sdss.ra.min()
decs_sim = np.random.rand(len(coo_twomass))*coo_sdss.dec.ptp() + coo_sdss.dec.min()
ras_sim, decs_sim


Ahora creamos un objeto `SkyCoord` a partir de estos puntos y lo cotejamos con `coo_sdss` tal y como hicimos anteriormente para 2MASS.

Observemos que no es necesario especificar explícitamente las unidades para `ras_sim` y `decs_sim`, porque ya son objetos con unidad `Angle` porque fueron creados desde `coo_sdss.ra`y `coo_sdss.dec`.

In [ ]:
coo_simulated = SkyCoord(ras_sim, decs_sim)  
idx_sim, d2d_sim, d3d_sim = coo_simulated.match_to_catalog_sky(coo_sdss)

In [ ]:
plt.hist(d2d_sim.arcsec, bins='auto', histtype='step', label='Simulated', linestyle='dashed')
plt.hist(d2d_sdss.arcsec, bins='auto', histtype='step', label='2MASS')
plt.xlabel('separation [arcsec]')
plt.legend(loc=0)
plt.tight_layout()

Muy bien, parece que las fuentes colocadas al azar están a un minuto de arco de distancia, así que probablemente podemos confiar en que nuestras primeras coincidencias (que estaban a un segundo de arco) son válidas.

## Explorando Gaia con Astroquery y el servicio TAP

In [ ]:
from astroquery.gaia import Gaia

In [ ]:
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print(table.get_qualified_name())

Antes de empezar con Gaia, investiguemos un poco sobre nuestro cúmulo globular de interés https://en.wikipedia.org/wiki/NGC_2808

https://archive.eso.org/dss/dss

In [ ]:
gc_name='ngc2808'

im_arcmin = 20
cutoutbaseurl = 'https://archive.eso.org/dss/dss/image'
endurl = '&Sky-Survey=DSS2-red&mime-type=download-gif&statsmode=WEBFORM'
query_string = urllib.parse.urlencode(dict(
                                     name=gc_name,
                                     x=im_arcmin, y=im_arcmin
                                          ))
url = cutoutbaseurl + '?' + query_string + endurl

urllib.request.urlretrieve(url, 'NGC2808_DSS2_cutout.jpg')

In [ ]:
IPython.display.Image('NGC2808_DSS2_cutout.jpg')

In [ ]:
ang_diam = 13*u.arcmin+0.8*u.arcsec

In [ ]:
ngc2808_center = SkyCoord.from_name('ngc 2808')
ngc2808_center

In [ ]:
meanpar=0.112 # Vasiliev 2021 edr3 https://arxiv.org/pdf/2102.09568.pdf 1e-2 error
dist=1*u.kpc/meanpar 
dist

In [ ]:
1*u.kpc/0.122

In [ ]:
1*u.kpc/0.102

In [ ]:
r_gc=(dist*ang_diam/2).to(u.kpc*u.rad)/u.rad
r_gc

In [ ]:
prange=0.5*u.kpc
parmax=1/(dist-prange)
parmax

In [ ]:
parmin=1/(dist+prange)
parmin

In [ ]:
ang_diam.to(u.deg)

https://gaia.aip.de/cms/documentation/cone-search/

In [ ]:
job2 = Gaia.launch_job_async("SELECT * \
FROM gaiaedr3.gaia_source \
WHERE CONTAINS(POINT('ICRS',gaiaedr3.gaia_source.ra,gaiaedr3.gaia_source.dec),CIRCLE('ICRS',138.01291667,-64.8635,0.22))=1 \
AND abs(parallax)>0.106 \
AND parallax IS NOT NULL \
AND parallax >0 \
AND abs(parallax)<0.119;", dump_to_file=True)

In [ ]:
j = job2.get_results()
print (j['source_id']) 

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(j['ra'],j['dec'])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(j['ra']*np.pi/180,j['dec']*np.pi/180,1000/j['parallax'])
ax.set_xlabel("ra (deg)")
ax.set_ylabel("dec (deg)")
ax.set_zlabel("r (pc)")

In [ ]:
plt.hist(1000/j['parallax'])
plt.axvline(1000/parmin.value,c='r')
plt.axvline(1000/parmax.value,c='r')

Query sin ADQL explícito: https://astroquery.readthedocs.io/en/latest/gaia/gaia.html

In [ ]:
j.columns

In [ ]:
bp_rp = j['bp_rp'].data
mg = j['phot_g_mean_mag'].data

In [ ]:
filt=~(np.isnan(mg.data) | np.isnan(bp_rp.data))
(~filt).sum()

In [ ]:
x=bp_rp.data[filt]
y=mg.data[filt]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6),dpi=150)
ax.scatter(x, y, alpha=1, s=10, color='k', zorder=0)
ax.invert_yaxis()
ax.set_xlabel(r'$G_{BP} - G_{RP}$')
ax.set_ylabel(r'$M_G$')
ax.set_title('Diagrama HR para NGC 2808')

https://www.researchgate.net/figure/The-observed-and-modeled-color-magnitude-diagrams-of-the-globular-cluster-NGC-2808-Yi_fig5_305273888

## Revisando inconsistencias en otras bases de datos

In [ ]:
from astroquery.simbad import Simbad

In [ ]:
customSimbad = Simbad()

customSimbad.get_votable_fields()

In [ ]:
customSimbad.add_votable_fields('plx')

In [ ]:
result_table = customSimbad.query_object("NGC 2808")
result_table

In [ ]:
result_table['PLX_VALUE']

In [ ]:
dist=1*u.kpc/(result_table['PLX_VALUE'][0])
dist

## Automatizando una búsqueda de Gaia

In [ ]:
from astropy.io.votable import parse_single_table


In [ ]:
votable = parse_single_table("lcc_simbad_votable.xml").to_table()

In [ ]:
first="SELECT * \
    FROM gaiadr2.gaia_source \
    WHERE "
stor="OR "
last=    "AND abs(parallax)>3.69 ;"

In [ ]:
(0.1*u.arcmin).to(u.deg)

In [ ]:
coun=0
for i,j in zip(votable['RA_d'],votable['DEC_d']):
    quer="CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS',%f,%f,0.00166))=1 "%(i,j)
    if coun==0:
        mainq=first+quer
    if coun>0:
        mainq+=stor+quer
    coun+=1
mainq+=last

In [ ]:
mainq

In [ ]:
coun

In [ ]:
job3=Gaia.launch_job(mainq, dump_to_file=True)

In [ ]:
j = job3.get_results()
print (j['source_id']) 

In [ ]:
j=j[j['parallax']>3]

In [ ]:
j.write('lcc.dat',format='ascii')

In [ ]:
tablelcc=Table.read('lcc.dat',format='ascii')
tablelcc=tablelcc[np.isfinite(tablelcc['parallax'])]
tablelcc=tablelcc[np.isfinite(tablelcc['radial_velocity'])]
print(len(tablelcc),np.isnan(tablelcc['pmdec']).sum(),np.isnan(tablelcc['pmra']).sum())

In [ ]:
def astrosol(j):
    x=1000/j['parallax']*np.cos(j['dec']*np.pi/180)*np.cos(j['ra']*np.pi/180)
    y=1000/j['parallax']*np.cos(j['dec']*np.pi/180)*np.sin(j['ra']*np.pi/180)
    z=1000/j['parallax']*np.sin(j['dec']*np.pi/180)
    vx=[]
    vy=[]
    vz=[]
    xa=[]
    ya=[]
    za=[]
    for i in range(len(j)):
        mdec=j['dec'][i]
        mra=j['ra'][i]
        mpar=j['parallax'][i]
        mpmra=j['pmra'][i]
        mpmdec=j['pmdec'][i]
        mvr=j['radial_velocity'][i]
        c1 = coord.ICRS(ra=mra*u.degree, dec=mdec*u.degree,
                    distance=(mpar*u.mas).to(u.pc, u.parallax()),
                    pm_ra_cosdec=mpmra*u.mas/u.yr,
                    pm_dec=mpmdec*u.mas/u.yr,
                    radial_velocity=mvr*u.km/u.s)
        gc1 = c1.transform_to(coord.Galactocentric)
        vx+=[gc1.v_x.value]
        vy+=[gc1.v_y.value]
        vz+=[gc1.v_z.value]
        xa+=[gc1.x.value]
        ya+=[gc1.y.value]
        za+=[gc1.z.value]
    vx=np.array(vx)
    vy=np.array(vy)
    vz=np.array(vz)
    return x,y,z,vx,vy,vz,xa,ya,za

In [ ]:
lccas=astrosol(tablelcc)
x2,y2,z2,vx2,vy2,vz2,xa2,ya2,za2=lccas

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
norm=10
ax.quiver(xa2,ya2,za2,vx2/norm,vy2/norm,vz2/norm,arrow_length_ratio=0.5,color='r')
ax.set_xlabel('x (pc)')
ax.set_ylabel('y (pc)')
ax.set_zlabel('z (pc)')